In [ ]:
!pip install panda3d


In [ ]:
!pip uninstall panda3d -y
!pip install panda3d==1.10.14


In [ ]:
from direct.showbase.ShowBase import ShowBase
from panda3d.core import ColorAttrib
from direct.task import Task


class TeslaLikeDemo(ShowBase):
    def __init__(self):
        super().__init__()
        self.disableMouse()

        # =============================
        # カメラ
        # =============================
        self.camera.setPos(-2.5, -20, 5)
        self.camera.lookAt(0, 0, 2)
        self.camLens.setFov(50)

        # =============================
        # 基本パラメータ
        # =============================
        self.car_width = 4.0
        self.lane_width = self.car_width * 2.6
        self.road_speed = 15.0
        self.segment_length = 2000

        # =============================
        # 白線X位置（3本）
        # =============================
        self.lane_lines_x = [
            -self.lane_width,   # 左端線
            0.0,                # 中央線
            self.lane_width     # 右端線
        ]

        # =============================
        # 車（左車線の中央）
        # =============================
        self.car = self.loader.loadModel("models/car/scene.gltf")
        self.car.reparentTo(self.render)

        # glTFは座標・向き・スケールがバラバラなので強制指定
        # glTFは座標・向き・スケールがバラバラなので強制指定
        self.car.clearTransform()
        self.car.setScale(0.04)   # サイズはそのまま
        #self.car.setPos(0, 0, 0)
        left_lane_center_x = -self.lane_width / 2.2
        self.car.setPos(left_lane_center_x, 0, -0.1)
        
        self.car.setP(90)         # ← これは上下軸補正（そのままでOK）
        self.car.setH(90)         # ← ★ これを追加：+X → +Y に向ける


        # デバッグ：ノード構造を表示
        self.car.ls()

        # =============================
        # 電柱
        # =============================
        self.pole_offset_x = self.lane_width * 2

        # =============================
        # 道路（左右対称・中央）
        # =============================
        self.road_width = self.pole_offset_x * 2

        self.roads = []
        for y in (0, self.segment_length):
            road = self.loader.loadModel("models/box")
            road.setScale(self.road_width, self.segment_length, 0.1)
            road.setPos(-self.road_width/2.0, y, -0.1)   # ← 絶対にずらさない
            road.setShaderOff()
            road.setLightOff()
            road.setAttrib(ColorAttrib.makeFlat((0.15, 0.15, 0.15, 1)))
            road.reparentTo(self.render)
            self.roads.append(road)

        # =============================
        # 白線
        # =============================
        self.lanes = []
        for y in (0, self.segment_length):
            for x in self.lane_lines_x:
                lane = self.loader.loadModel("models/box")
                lane.setScale(0.15, self.segment_length, 0.05)
                lane.setPos(x, y, -0.01)
                lane.setShaderOff()
                lane.setLightOff()
                lane.setAttrib(ColorAttrib.makeFlat((0.9, 0.9, 0.9, 1)))
                lane.reparentTo(self.render)
                self.lanes.append(lane)

        # =============================
        # 電柱
        # =============================
        self.poles = []
        self.pole_spacing = 10.0
        pole_count = int(self.segment_length / self.pole_spacing)

        for i in range(pole_count):
            for x in (-self.pole_offset_x, self.pole_offset_x):
                pole = self.loader.loadModel("models/box")
                pole.setScale(0.3, 0.3, 5.0)
                pole.setPos(x, i * self.pole_spacing, -1.0)
                pole.setShaderOff()
                pole.setLightOff()
                pole.setAttrib(ColorAttrib.makeFlat((0.2, 0.2, 0.2, 1)))
                pole.reparentTo(self.render)
                self.poles.append(pole)

        self.taskMgr.add(self.update, "update")

    def update(self, task: Task):
        dt = globalClock.getDt()
        move = self.road_speed * dt

        for road in self.roads:
            road.setY(road.getY() - move)
            if road.getY() < -self.segment_length:
                road.setY(road.getY() + self.segment_length * 2)

        for lane in self.lanes:
            lane.setY(lane.getY() - move)
            if lane.getY() < -self.segment_length:
                lane.setY(lane.getY() + self.segment_length * 2)

        for pole in self.poles:
            pole.setY(pole.getY() - move)
            if pole.getY() < -10:
                pole.setY(pole.getY() + self.segment_length)

        return Task.cont


app = TeslaLikeDemo()
app.run()


In [ ]:
from direct.showbase.ShowBase import ShowBase
from panda3d.core import ColorAttrib, TextNode
from direct.task import Task
from direct.gui.DirectGui import DirectButton
from direct.gui import DirectGuiGlobals as DGG


class TeslaLikeDemo(ShowBase):
    def __init__(self):
        super().__init__()
        self.disableMouse()

        # =============================
        # カメラ
        # =============================
        self.camera.setPos(-2.5, -20, 5)
        self.camera.lookAt(0, 0, 2)
        self.camLens.setFov(50)

        # =============================
        # 基本パラメータ
        # =============================
        self.car_width = 4.0
        self.lane_width = self.car_width * 2.6
        self.road_speed = 15.0
        self.segment_length = 2000

        # =============================
        # 白線X位置
        # =============================
        self.lane_lines_x = [-self.lane_width, 0.0, self.lane_width]

        # =============================
        # 車（操作）
        # =============================
        self.car = self.loader.loadModel("models/car/scene.gltf")
        self.car.reparentTo(self.render)
        self.car.clearTransform()
        self.car.setScale(0.04)
        self.car.setPos(-self.lane_width / 2.2, 0, -0.1)
        self.car.setP(90)
        self.car.setH(90)

        # =============================
        # ゴースト車（基準）
        # =============================
        self.car1 = self.loader.loadModel("models/car/scene.gltf")
        self.car1.reparentTo(self.render)
        self.car1.clearTransform()
        self.car1.setScale(0.04)
        self.car1.setPos(-self.lane_width / 2.2, 0.041, -0.141)
        self.car1.setP(90)
        self.car1.setH(90)
        self.car1.setAttrib(ColorAttrib.makeFlat((0.4, 0.4, 0.4, 1)))

        # =============================
        # 横移動
        # =============================
        self.car_speed_x = 6.0
        self.move_left = False
        self.move_right = False
        self.min_x = -self.lane_width
        self.max_x = self.lane_width

        # =============================
        # 自動補正
        # =============================
        self.auto_align = False
        self.align_speed = 1.0
        self.dead_zone = 0.15

        # =============================
        # キーボード
        # =============================
        self.accept("arrow_left", self.set_move_left, [True])
        self.accept("arrow_left-up", self.set_move_left, [False])
        self.accept("arrow_right", self.set_move_right, [True])
        self.accept("arrow_right-up", self.set_move_right, [False])

        # =============================
        # 左上ボタン（サイズ固定）
        # =============================
        btn_height = 0.55          # 上下の半分
        text_scale = 0.6

        btn_scale = 0.10
        lr_frame = (-1.5, 1.5, -btn_height, btn_height)
        
        self.btn_left = DirectButton(
            text="Left",
            scale=btn_scale,
            pos=(-0.95, 0, 0.85),
            frameSize=lr_frame,
            relief=DGG.FLAT,
            text_align=TextNode.ACenter,
            text_scale=text_scale,
        )
        self.btn_left.bind(DGG.B1PRESS, lambda e: self.set_move_left(True))
        self.btn_left.bind(DGG.B1RELEASE, lambda e: self.set_move_left(False))
        
        self.btn_right = DirectButton(
            text="Right",
            scale=btn_scale,
            pos=(-0.55, 0, 0.85),
            frameSize=lr_frame,
            relief=DGG.FLAT,
            text_align=TextNode.ACenter,
            text_scale=text_scale,
        )
        self.btn_right.bind(DGG.B1PRESS, lambda e: self.set_move_right(True))
        self.btn_right.bind(DGG.B1RELEASE, lambda e: self.set_move_right(False))



        # =============================
        # AUTO ALIGN ボタン（右上）
        # =============================
        align_frame = (-2.2, 2.2, -btn_height, btn_height)

        self.btn_align = DirectButton(
            text="Auto Align",
            scale=0.09,
            pos=(1.05, 0, 0.85),
            frameSize=align_frame,
            relief=DGG.FLAT,
            text_align=TextNode.ACenter,
            text_scale=text_scale,
            command=self.enable_auto_align
        )

        # =============================
        # 警告テキスト
        # =============================
        self.warning_text = TextNode("warning")
        self.warning_text.setAlign(TextNode.ACenter)
        self.warning_text.setTextColor(1, 0.2, 0.2, 1)
        self.warning_text.setTextScale(0.12)

        self.warning_np = aspect2d.attachNewNode(self.warning_text)
        self.warning_np.setPos(0, 0, 0.55)
        self.warning_np.hide()

        # =============================
        # 道路
        # =============================
        self.pole_offset_x = self.lane_width * 2
        self.road_width = self.pole_offset_x * 2

        self.roads = []
        for y in (0, self.segment_length):
            road = self.loader.loadModel("models/box")
            road.setScale(self.road_width, self.segment_length, 0.1)
            road.setPos(-self.road_width / 2, y, -0.1)
            road.setAttrib(ColorAttrib.makeFlat((0.15, 0.15, 0.15, 1)))
            road.setShaderOff()
            road.setLightOff()
            road.reparentTo(self.render)
            self.roads.append(road)

        # =============================
        # 白線
        # =============================
        self.lanes = []
        for y in (0, self.segment_length):
            for x in self.lane_lines_x:
                lane = self.loader.loadModel("models/box")
                lane.setScale(0.15, self.segment_length, 0.05)
                lane.setPos(x, y, -0.01)
                lane.setAttrib(ColorAttrib.makeFlat((0.9, 0.9, 0.9, 1)))
                lane.setShaderOff()
                lane.setLightOff()
                lane.reparentTo(self.render)
                self.lanes.append(lane)

        # =============================
        # 電柱
        # =============================
        self.poles = []
        self.pole_spacing = 10.0
        for i in range(int(self.segment_length / self.pole_spacing)):
            for x in (-self.pole_offset_x, self.pole_offset_x):
                pole = self.loader.loadModel("models/box")
                pole.setScale(0.3, 0.3, 5.0)
                pole.setPos(x, i * self.pole_spacing, -1.0)
                pole.setAttrib(ColorAttrib.makeFlat((0.2, 0.2, 0.2, 1)))
                pole.setShaderOff()
                pole.setLightOff()
                pole.reparentTo(self.render)
                self.poles.append(pole)

        self.taskMgr.add(self.update, "update")

    # =============================
    # 入力
    # =============================
    def set_move_left(self, flag):
        self.move_left = flag

    def set_move_right(self, flag):
        self.move_right = flag

    def enable_auto_align(self):
        self.auto_align = True

    # =============================
    # 更新
    # =============================
    def update(self, task: Task):
        dt = globalClock.getDt()
        move = self.road_speed * dt

        # --- 環境スクロール ---
        for road in self.roads:
            road.setY(road.getY() - move)
            if road.getY() < -self.segment_length:
                road.setY(road.getY() + self.segment_length * 2)

        for lane in self.lanes:
            lane.setY(lane.getY() - move)
            if lane.getY() < -self.segment_length:
                lane.setY(lane.getY() + self.segment_length * 2)

        for pole in self.poles:
            pole.setY(pole.getY() - move)
            if pole.getY() < -10:
                pole.setY(pole.getY() + self.segment_length)

        # --- 車の左右移動 ---
        x = self.car.getX()
        if self.move_left:
            x -= self.car_speed_x * dt
        if self.move_right:
            x += self.car_speed_x * dt

        # --- 自動補正 ---
        dx = self.car.getX() - self.car1.getX()
        if self.auto_align:
            if abs(dx) > self.dead_zone:
                x -= dx * self.align_speed * dt
            else:
                x = self.car1.getX()
                self.auto_align = False

        self.car.setX(max(self.min_x, min(self.max_x, x)))

        # --- 警告 ---
        if abs(dx) > self.dead_zone:
            if dx > 0:
                # 右に寄っている → 赤
                self.warning_text.setText("WARNING: Vehicle drifting to the Right")
                self.warning_text.setTextColor(0.7, 0.3, 0.4, 1)  # 赤
            else:
                # 左に寄っている → 青
                self.warning_text.setText("WARNING: Vehicle drifting to the Left")
                self.warning_text.setTextColor(0.3, 0.5, 0.7, 1)  # 青
        
            self.warning_np.show()
        else:
            self.warning_np.hide()


        return Task.cont


app = TeslaLikeDemo()
app.run()


In [ ]:
from direct.showbase.ShowBase import ShowBase
from panda3d.core import ColorAttrib, TextNode
from direct.task import Task
from direct.gui.DirectGui import DirectButton
from direct.gui import DirectGuiGlobals as DGG
import random


class TeslaLikeDemo(ShowBase):
    def __init__(self):
        super().__init__()
        self.disableMouse()

        # =============================
        # カメラ
        # =============================
        self.camera.setPos(-2.5, -20, 5)
        self.camera.lookAt(0, 0, 2)
        self.camLens.setFov(50)

        # =============================
        # 基本パラメータ
        # =============================
        self.car_width = 4.0
        self.lane_width = self.car_width * 2.6
        self.road_speed = 30.0
        self.segment_length = 2000

        # =============================
        # 白線X位置
        # =============================
        self.lane_lines_x = [-self.lane_width, 0.0, self.lane_width]

        # =============================
        # 操作車
        # =============================
        self.car = self.loader.loadModel("models/car/scene.gltf")
        self.car.reparentTo(self.render)
        self.car.clearTransform()
        self.car.setScale(0.04)
        self.car.setPos(-self.lane_width / 2.2, 0, -0.1)
        self.car.setP(90)
        self.car.setH(90)

        # =============================
        # ゴースト車（基準）
        # =============================
        self.car1 = self.loader.loadModel("models/car/scene.gltf")
        self.car1.reparentTo(self.render)
        self.car1.clearTransform()
        self.car1.setScale(0.04)
        self.car1.setPos(-self.lane_width / 2.2, 0.041, -0.141)
        self.car1.setP(90)
        self.car1.setH(90)
        self.car1.setAttrib(ColorAttrib.makeFlat((0.4, 0.4, 0.4, 1)))

        # =============================
        # 横移動
        # =============================
        self.car_speed_x = 6.0
        self.move_left = False
        self.move_right = False
        self.min_x = -self.lane_width
        self.max_x = self.lane_width

        # =============================
        # 自動補正
        # =============================
        self.auto_align = False
        self.align_speed = 1.0
        self.dead_zone = 0.15

        # =============================
        # キーボード
        # =============================
        self.accept("arrow_left", self.set_move_left, [True])
        self.accept("arrow_left-up", self.set_move_left, [False])
        self.accept("arrow_right", self.set_move_right, [True])
        self.accept("arrow_right-up", self.set_move_right, [False])

        # =============================
        # ボタン
        # =============================
        btn_height = 0.55
        text_scale = 0.6
        btn_scale = 0.10
        lr_frame = (-1.5, 1.5, -btn_height, btn_height)

        self.btn_left = DirectButton(
            text="Left",
            scale=btn_scale,
            pos=(-0.95, 0, 0.85),
            frameSize=lr_frame,
            relief=DGG.FLAT,
            text_align=TextNode.ACenter,
            text_scale=text_scale,
        )
        self.btn_left.bind(DGG.B1PRESS, lambda e: self.set_move_left(True))
        self.btn_left.bind(DGG.B1RELEASE, lambda e: self.set_move_left(False))

        self.btn_right = DirectButton(
            text="Right",
            scale=btn_scale,
            pos=(-0.55, 0, 0.85),
            frameSize=lr_frame,
            relief=DGG.FLAT,
            text_align=TextNode.ACenter,
            text_scale=text_scale,
        )
        self.btn_right.bind(DGG.B1PRESS, lambda e: self.set_move_right(True))
        self.btn_right.bind(DGG.B1RELEASE, lambda e: self.set_move_right(False))

        self.btn_align = DirectButton(
            text="Auto Align",
            scale=0.09,
            pos=(1.05, 0, 0.85),
            frameSize=(-2.2, 2.2, -btn_height, btn_height),
            relief=DGG.FLAT,
            text_align=TextNode.ACenter,
            text_scale=text_scale,
            command=self.enable_auto_align
        )

        # =============================
        # 警告テキスト
        # =============================
        self.warning_text = TextNode("warning")
        self.warning_text.setAlign(TextNode.ACenter)
        self.warning_text.setTextScale(0.12)
        self.warning_np = aspect2d.attachNewNode(self.warning_text)
        self.warning_np.setPos(0, 0, 0.55)
        self.warning_np.hide()

        # =============================
        # 道路
        # =============================
        self.pole_offset_x = self.lane_width * 2
        self.road_width = self.pole_offset_x * 2

        self.roads = []
        for y in (0, self.segment_length):
            road = self.loader.loadModel("models/box")
            road.setScale(self.road_width, self.segment_length, 0.1)
            road.setPos(-self.road_width / 2, y, -0.1)
            road.setAttrib(ColorAttrib.makeFlat((0.15, 0.15, 0.15, 1)))
            road.setShaderOff()
            road.setLightOff()
            road.reparentTo(self.render)
            self.roads.append(road)

        # =============================
        # 白線
        # =============================
        self.lanes = []
        for y in (0, self.segment_length):
            for x in self.lane_lines_x:
                lane = self.loader.loadModel("models/box")
                lane.setScale(0.15, self.segment_length, 0.05)
                lane.setPos(x, y, -0.01)
                lane.setAttrib(ColorAttrib.makeFlat((0.9, 0.9, 0.9, 1)))
                lane.setShaderOff()
                lane.setLightOff()
                lane.reparentTo(self.render)
                self.lanes.append(lane)

        # =============================
        # 電柱
        # =============================
        self.poles = []
        self.pole_spacing = 10.0
        for i in range(int(self.segment_length / self.pole_spacing)):
            for x in (-self.pole_offset_x, self.pole_offset_x):
                pole = self.loader.loadModel("models/box")
                pole.setScale(0.3, 0.3, 5.0)
                pole.setPos(x, i * self.pole_spacing, -1.0)
                pole.setAttrib(ColorAttrib.makeFlat((0.2, 0.2, 0.2, 1)))
                pole.setShaderOff()
                pole.setLightOff()
                pole.reparentTo(self.render)
                self.poles.append(pole)

        # =============================
        # 反対車線の車
        # =============================
        self.oncoming_cars = []
        y = 0
        for _ in range(8):
            car = self.loader.loadModel("models/car/scene.gltf")
            car.reparentTo(self.render)
            car.clearTransform()
            car.setScale(0.04)
            car.setPos(self.lane_width / 2.2, y + random.uniform(10, 50), -0.1)
            car.setP(90)
            car.setH(-90)
            car.setAttrib(ColorAttrib.makeFlat((0.3, 1, 1, 1)))
            self.oncoming_cars.append(car)
            y += random.uniform(140, 220)

        self.taskMgr.add(self.update, "update")

    # =============================
    # 入力
    # =============================
    def set_move_left(self, flag):
        self.move_left = flag

    def set_move_right(self, flag):
        self.move_right = flag

    def enable_auto_align(self):
        self.auto_align = True

    # =============================
    # 更新
    # =============================
    def update(self, task: Task):
        dt = globalClock.getDt()
        move = self.road_speed * dt

        for road in self.roads:
            road.setY(road.getY() - move)
            if road.getY() < -self.segment_length:
                road.setY(road.getY() + self.segment_length * 2)

        for lane in self.lanes:
            lane.setY(lane.getY() - move)
            if lane.getY() < -self.segment_length:
                lane.setY(lane.getY() + self.segment_length * 2)

        for pole in self.poles:
            pole.setY(pole.getY() - move)
            if pole.getY() < -10:
                pole.setY(pole.getY() + self.segment_length)

        for car in self.oncoming_cars:
            car.setY(car.getY() - move)
            if car.getY() < -20:
                car.setY(self.segment_length)

        x = self.car.getX()
        if self.move_left:
            x -= self.car_speed_x * dt
        if self.move_right:
            x += self.car_speed_x * dt

        dx = self.car.getX() - self.car1.getX()
        if self.auto_align:
            if abs(dx) > self.dead_zone:
                x -= dx * self.align_speed * dt
            else:
                x = self.car1.getX()
                self.auto_align = False

        self.car.setX(max(self.min_x, min(self.max_x, x)))

        if abs(dx) > self.dead_zone:
            if dx > 0:
                self.warning_text.setText("WARNING: Vehicle drifting to the Right")
                self.warning_text.setTextColor(0.7, 0.3, 0.4, 1)
            else:
                self.warning_text.setText("WARNING: Vehicle drifting to the Left")
                self.warning_text.setTextColor(0.3, 0.5, 0.7, 1)
            self.warning_np.show()
        else:
            self.warning_np.hide()

        return Task.cont


app = TeslaLikeDemo()
app.run()


In [ ]:
from direct.showbase.ShowBase import ShowBase
from panda3d.core import ColorAttrib, TextNode, LineSegs
from direct.task import Task
from direct.gui.DirectGui import DirectButton
from direct.gui import DirectGuiGlobals as DGG
import random


class TeslaLikeDemo(ShowBase):
    def __init__(self):
        super().__init__()
        self.disableMouse()

        # =============================
        # カメラ
        # =============================
        self.camera.setPos(-2.5, -65, 40)
        self.camera.lookAt(0, 20, 5)
        self.camLens.setFov(50)

        # =============================
        # 基本パラメータ
        # =============================
        self.car_width = 4.0
        self.lane_width = self.car_width * 2.6
        self.road_speed = 30.0
        self.segment_length = 2000

        # =============================
        # 白線X位置
        # =============================
        self.lane_lines_x = [-self.lane_width, 0.0, self.lane_width]

        # =============================
        # 操作車
        # =============================
        self.car = self.loader.loadModel("models/car/scene.gltf")
        self.car.reparentTo(self.render)
        self.car.clearTransform()
        self.car.setScale(0.04)
        self.car.setPos(-self.lane_width / 2.2, 0, -0.1)
        self.car.setP(90)
        self.car.setH(90)

        # =============================
        # ゴースト車（基準）
        # =============================
        self.car1 = self.loader.loadModel("models/car/scene.gltf")
        self.car1.reparentTo(self.render)
        self.car1.clearTransform()
        self.car1.setScale(0.04)
        self.car1.setPos(-self.lane_width / 2.2, 0.041, -0.141)
        self.car1.setP(90)
        self.car1.setH(90)
        self.car1.setAttrib(ColorAttrib.makeFlat((0.4, 0.4, 0.4, 1)))

        # =============================
        # 横移動
        # =============================
        self.car_speed_x = 6.0
        self.move_left = False
        self.move_right = False
        self.min_x = -self.lane_width
        self.max_x = self.lane_width

        # =============================
        # 自動補正
        # =============================
        self.auto_align = False
        self.align_speed = 1.0
        self.dead_zone = 0.15

        # =============================
        # キーボード
        # =============================
        self.accept("arrow_left", self.set_move_left, [True])
        self.accept("arrow_left-up", self.set_move_left, [False])
        self.accept("arrow_right", self.set_move_right, [True])
        self.accept("arrow_right-up", self.set_move_right, [False])

        # =============================
        # ボタン
        # =============================
        btn_height = 0.55
        text_scale = 0.6
        btn_scale = 0.10
        lr_frame = (-1.5, 1.5, -btn_height, btn_height)

        self.btn_left = DirectButton(
            text="Left",
            scale=btn_scale,
            pos=(-0.95, 0, 0.85),
            frameSize=lr_frame,
            relief=DGG.FLAT,
            text_align=TextNode.ACenter,
            text_scale=text_scale,
        )
        self.btn_left.bind(DGG.B1PRESS, lambda e: self.set_move_left(True))
        self.btn_left.bind(DGG.B1RELEASE, lambda e: self.set_move_left(False))

        self.btn_right = DirectButton(
            text="Right",
            scale=btn_scale,
            pos=(-0.55, 0, 0.85),
            frameSize=lr_frame,
            relief=DGG.FLAT,
            text_align=TextNode.ACenter,
            text_scale=text_scale,
        )
        self.btn_right.bind(DGG.B1PRESS, lambda e: self.set_move_right(True))
        self.btn_right.bind(DGG.B1RELEASE, lambda e: self.set_move_right(False))

        self.btn_align = DirectButton(
            text="Auto Align",
            scale=0.09,
            pos=(1.05, 0, 0.85),
            frameSize=(-2.2, 2.2, -btn_height, btn_height),
            relief=DGG.FLAT,
            text_align=TextNode.ACenter,
            text_scale=text_scale,
            command=self.enable_auto_align
        )

        # =============================
        # 警告テキスト
        # =============================
        self.warning_text = TextNode("warning")
        self.warning_text.setAlign(TextNode.ACenter)
        self.warning_text.setTextScale(0.12)
        self.warning_np = aspect2d.attachNewNode(self.warning_text)
        self.warning_np.setPos(0, 0, 0.55)
        self.warning_np.hide()

        # =============================
        # 道路
        # =============================
        self.pole_offset_x = self.lane_width * 2
        self.road_width = self.pole_offset_x * 2

        self.roads = []
        for y in (0, self.segment_length):
            road = self.loader.loadModel("models/box")
            road.setScale(self.road_width, self.segment_length, 0.1)
            road.setPos(-self.road_width / 2, y, -0.1)
            road.setAttrib(ColorAttrib.makeFlat((0.15, 0.15, 0.15, 1)))
            road.reparentTo(self.render)
            self.roads.append(road)

        # =============================
        # 白線
        # =============================
        self.lanes = []
        for y in (0, self.segment_length):
            for x in self.lane_lines_x:
                lane = self.loader.loadModel("models/box")
                lane.setScale(0.15, self.segment_length, 0.05)
                lane.setPos(x, y, -0.01)
                lane.setAttrib(ColorAttrib.makeFlat((0.9, 0.9, 0.9, 1)))
                lane.reparentTo(self.render)
                self.lanes.append(lane)

        # =============================
        # 電柱
        # =============================
        self.poles = []
        self.pole_spacing = 10.0
        for i in range(int(self.segment_length / self.pole_spacing)):
            for x in (-self.pole_offset_x, self.pole_offset_x):
                pole = self.loader.loadModel("models/box")
                pole.setScale(0.3, 0.3, 5.0)
                pole.setPos(x, i * self.pole_spacing, -1.0)
                pole.setAttrib(ColorAttrib.makeFlat((0.2, 0.2, 0.2, 1)))
                pole.reparentTo(self.render)
                self.poles.append(pole)

        # =============================
        # 反対車線の車
        # =============================
        self.oncoming_cars = []
        y = 0
        for _ in range(8):
            car = self.loader.loadModel("models/car/scene.gltf")
            car.reparentTo(self.render)
            car.setScale(0.04)
            car.setPos(self.lane_width / 2.2, y + random.uniform(10, 50), -0.1)
            car.setP(90)
            car.setH(-90)
            car.setAttrib(ColorAttrib.makeFlat((0.3, 1, 1, 1)))
            self.oncoming_cars.append(car)
            y += random.uniform(140, 220)

        # =============================
        # ナビビーム
        # =============================
        self.nav_root = self.render.attachNewNode("nav_root")
        self.nav_np = None
        self.beam_length = self.car_width * 2.0
        self.target_lane_x = -self.lane_width / 2.2

        self.taskMgr.add(self.update, "update")

        # =============================
        # ナビビーム基準
        # =============================
        self.base_lane_x = self.car.getX()   # ← 初期位置を記憶
        self.beam_length = self.car_width * 2.0


    # =============================
    # 入力
    # =============================
    def set_move_left(self, flag):
        self.move_left = flag

    def set_move_right(self, flag):
        self.move_right = flag

    def enable_auto_align(self):
        self.auto_align = True

    # =============================
    # ナビビーム更新
    # =============================
    def update_nav_beam(self):
        if self.nav_np:
            self.nav_np.removeNode()

        segs = LineSegs()
        segs.setThickness(8)
        segs.setColor(0.3, 0.4, 0.5, 0.8)

        cx = self.car.getX()
        cy = self.car.getY()
        cz = self.car.getZ() + 0.3

        # 始点：車の先頭
        start = (cx, cy + 1.5, cz)

        # 終点：初期Xのまま前方
        end = (
            self.base_lane_x,
            cy + self.beam_length + 80,
            cz
        )

        # ズレ量（ここが歪みの正体）
        dx = self.base_lane_x - cx
        curve = dx * 0.7

        mid1 = (cx + curve * 0.3, cy + self.beam_length * 0.3, cz)
        mid2 = (cx + curve * 0.7, cy + self.beam_length * 0.7, cz)

        segs.moveTo(start)
        segs.drawTo(mid1)
        segs.drawTo(mid2)
        segs.drawTo(end)

        self.nav_np = self.nav_root.attachNewNode(segs.create())


    # =============================
    # 更新
    # =============================
    def update(self, task: Task):
        dt = globalClock.getDt()
        move = self.road_speed * dt

        for road in self.roads:
            road.setY(road.getY() - move)
            if road.getY() < -self.segment_length:
                road.setY(road.getY() + self.segment_length * 2)

        for lane in self.lanes:
            lane.setY(lane.getY() - move)
            if lane.getY() < -self.segment_length:
                lane.setY(lane.getY() + self.segment_length * 2)

        for pole in self.poles:
            pole.setY(pole.getY() - move)
            if pole.getY() < -10:
                pole.setY(pole.getY() + self.segment_length)

        for car in self.oncoming_cars:
            car.setY(car.getY() - move)
            if car.getY() < -20:
                car.setY(self.segment_length)

        x = self.car.getX()
        if self.move_left:
            x -= self.car_speed_x * dt
        if self.move_right:
            x += self.car_speed_x * dt

        dx = self.car.getX() - self.car1.getX()
        if self.auto_align:
            if abs(dx) > self.dead_zone:
                x -= dx * self.align_speed * dt
            else:
                x = self.car1.getX()
                self.auto_align = False

        self.car.setX(max(self.min_x, min(self.max_x, x)))

        if abs(dx) > self.dead_zone:
            if dx > 0:
                self.warning_text.setText("WARNING: Vehicle drifting to the Right")
                self.warning_text.setTextColor(0.8, 0.3, 0.3, 1)
            else:
                self.warning_text.setText("WARNING: Vehicle drifting to the Left")
                self.warning_text.setTextColor(0.3, 0.5, 0.9, 1)
            self.warning_np.show()
        else:
            self.warning_np.hide()

        self.update_nav_beam()
        return Task.cont


app = TeslaLikeDemo()
app.run()


In [ ]:
from direct.showbase.ShowBase import ShowBase
from panda3d.core import ColorAttrib, TextNode, LineSegs
from direct.task import Task
from direct.gui.DirectGui import DirectButton
from direct.gui import DirectGuiGlobals as DGG
import random


class TeslaLikeDemo(ShowBase):
    def __init__(self):
        super().__init__()
        self.disableMouse()

        # =============================
        # カメラ（初期：一番遠い）
        # =============================
        self.camera.setPos(-2.5, -65, 40)
        self.camera.lookAt(0, 20, 5)
        self.camLens.setFov(50)

        # =============================
        # ズーム用パラメータ
        # =============================
        self.zoom = 0.0          # 現在値
        self.target_zoom = 0.0   # 目標値
        self.zoom_step = 0.1
        self.zoom_smooth = 6.0   # ← 大きいほどキビキビ

        self.cam_far_pos = (-2.5, -65, 40)
        self.cam_far_look = (0, 20, 5)

        self.cam_near_pos = (-2.5, -20, 5)
        self.cam_near_look = (0, 0, 2)

        self.accept("wheel_up", self.zoom_in)
        self.accept("wheel_down", self.zoom_out)

        # =============================
        # 基本パラメータ
        # =============================
        self.car_width = 4.0
        self.lane_width = self.car_width * 2.6
        self.road_speed = 30.0
        self.segment_length = 2000

        # =============================
        # 白線X位置
        # =============================
        self.lane_lines_x = [-self.lane_width, 0.0, self.lane_width]

        # =============================
        # 操作車
        # =============================
        self.car = self.loader.loadModel("models/car/scene.gltf")
        self.car.reparentTo(self.render)
        self.car.clearTransform()
        self.car.setScale(0.04)
        self.car.setPos(-self.lane_width / 2.2, 0, -0.1)
        self.car.setP(90)
        self.car.setH(90)

        # =============================
        # ゴースト車
        # =============================
        self.car1 = self.loader.loadModel("models/car/scene.gltf")
        self.car1.reparentTo(self.render)
        self.car1.clearTransform()
        self.car1.setScale(0.04)
        self.car1.setPos(-self.lane_width / 2.2, 0.041, -0.141)
        self.car1.setP(90)
        self.car1.setH(90)
        self.car1.setAttrib(ColorAttrib.makeFlat((0.4, 0.4, 0.4, 1)))

        # =============================
        # 横移動
        # =============================
        self.car_speed_x = 6.0
        self.move_left = False
        self.move_right = False
        self.min_x = -self.lane_width
        self.max_x = self.lane_width

        # =============================
        # 自動補正
        # =============================
        self.auto_align = False
        self.align_speed = 1.0
        self.dead_zone = 0.15

        # =============================
        # キーボード
        # =============================
        self.accept("arrow_left", self.set_move_left, [True])
        self.accept("arrow_left-up", self.set_move_left, [False])
        self.accept("arrow_right", self.set_move_right, [True])
        self.accept("arrow_right-up", self.set_move_right, [False])

        # =============================
        # ボタン
        # =============================
        btn_height = 0.55
        text_scale = 0.6
        btn_scale = 0.10
        lr_frame = (-1.5, 1.5, -btn_height, btn_height)

        self.btn_left = DirectButton(
            text="Left",
            scale=btn_scale,
            pos=(-0.95, 0, 0.85),
            frameSize=lr_frame,
            relief=DGG.FLAT,
            text_align=TextNode.ACenter,
            text_scale=text_scale,
        )
        self.btn_left.bind(DGG.B1PRESS, lambda e: self.set_move_left(True))
        self.btn_left.bind(DGG.B1RELEASE, lambda e: self.set_move_left(False))

        self.btn_right = DirectButton(
            text="Right",
            scale=btn_scale,
            pos=(-0.55, 0, 0.85),
            frameSize=lr_frame,
            relief=DGG.FLAT,
            text_align=TextNode.ACenter,
            text_scale=text_scale,
        )
        self.btn_right.bind(DGG.B1PRESS, lambda e: self.set_move_right(True))
        self.btn_right.bind(DGG.B1RELEASE, lambda e: self.set_move_right(False))

        self.btn_align = DirectButton(
            text="Auto Align",
            scale=0.09,
            pos=(1.05, 0, 0.85),
            frameSize=(-2.2, 2.2, -btn_height, btn_height),
            relief=DGG.FLAT,
            text_align=TextNode.ACenter,
            text_scale=text_scale,
            command=self.enable_auto_align
        )

        # =============================
        # 警告テキスト
        # =============================
        self.warning_text = TextNode("warning")
        self.warning_text.setAlign(TextNode.ACenter)
        self.warning_text.setTextScale(0.12)
        self.warning_np = aspect2d.attachNewNode(self.warning_text)
        self.warning_np.setPos(0, 0, 0.55)
        self.warning_np.hide()

        # =============================
        # 道路
        # =============================
        self.pole_offset_x = self.lane_width * 2
        self.road_width = self.pole_offset_x * 2

        self.roads = []
        for y in (0, self.segment_length):
            road = self.loader.loadModel("models/box")
            road.setScale(self.road_width, self.segment_length, 0.1)
            road.setPos(-self.road_width / 2, y, -0.1)
            road.setAttrib(ColorAttrib.makeFlat((0.15, 0.15, 0.15, 1)))
            road.reparentTo(self.render)
            self.roads.append(road)

        # =============================
        # 白線
        # =============================
        self.lanes = []
        for y in (0, self.segment_length):
            for x in self.lane_lines_x:
                lane = self.loader.loadModel("models/box")
                lane.setScale(0.15, self.segment_length, 0.05)
                lane.setPos(x, y, -0.01)
                lane.setAttrib(ColorAttrib.makeFlat((0.9, 0.9, 0.9, 1)))
                lane.reparentTo(self.render)
                self.lanes.append(lane)

        # =============================
        # 電柱
        # =============================
        self.poles = []
        self.pole_spacing = 10.0
        for i in range(int(self.segment_length / self.pole_spacing)):
            for x in (-self.pole_offset_x, self.pole_offset_x):
                pole = self.loader.loadModel("models/box")
                pole.setScale(0.3, 0.3, 5.0)
                pole.setPos(x, i * self.pole_spacing, -1.0)
                pole.setAttrib(ColorAttrib.makeFlat((0.2, 0.2, 0.2, 1)))
                pole.reparentTo(self.render)
                self.poles.append(pole)

        # =============================
        # 反対車線の車
        # =============================
        self.oncoming_cars = []
        y = 0
        for _ in range(11):
            car = self.loader.loadModel("models/car/scene.gltf")
            car.reparentTo(self.render)
            car.setScale(0.04)
            car.setPos(self.lane_width / 2.2, y + random.uniform(10, 50), -0.1)
            car.setP(90)
            car.setH(-90)
            car.setAttrib(ColorAttrib.makeFlat((0.3, 1, 1, 1)))
            self.oncoming_cars.append(car)
            y += random.uniform(140, 220)

        # =============================
        # ナビビーム
        # =============================
        self.nav_root = self.render.attachNewNode("nav_root")
        self.nav_np = None
        self.base_lane_x = self.car.getX()
        self.beam_length = self.car_width * 2.0

        self.taskMgr.add(self.update, "update")

    # =============================
    # ズーム操作
    # =============================
    def zoom_in(self):
        self.target_zoom = min(1.0, self.target_zoom + self.zoom_step)

    def zoom_out(self):
        self.target_zoom = max(0.0, self.target_zoom - self.zoom_step)

    def update_camera(self, dt):
        self.zoom += (self.target_zoom - self.zoom) * min(1.0, dt * self.zoom_smooth)

        t = self.zoom
        px = self.cam_far_pos[0] * (1 - t) + self.cam_near_pos[0] * t
        py = self.cam_far_pos[1] * (1 - t) + self.cam_near_pos[1] * t
        pz = self.cam_far_pos[2] * (1 - t) + self.cam_near_pos[2] * t
        self.camera.setPos(px, py, pz)

        lx = self.cam_far_look[0] * (1 - t) + self.cam_near_look[0] * t
        ly = self.cam_far_look[1] * (1 - t) + self.cam_near_look[1] * t
        lz = self.cam_far_look[2] * (1 - t) + self.cam_near_look[2] * t
        self.camera.lookAt(lx, ly, lz)

    # =============================
    # 入力
    # =============================
    def set_move_left(self, flag):
        self.move_left = flag

    def set_move_right(self, flag):
        self.move_right = flag

    def enable_auto_align(self):
        self.auto_align = True

    # =============================
    # ナビビーム
    # =============================
    def update_nav_beam(self):
        if self.nav_np:
            self.nav_np.removeNode()

        segs = LineSegs()
        segs.setThickness(8)
        segs.setColor(0.3, 0.4, 0.5, 0.8)

        cx, cy, cz = self.car.getX(), self.car.getY(), self.car.getZ() + 0.3
        start = (cx, cy + 1.5, cz)
        end = (self.base_lane_x, cy + self.beam_length + 80, cz)

        dx = self.base_lane_x - cx
        curve = dx * 0.7
        mid1 = (cx + curve * 0.3, cy + self.beam_length * 0.3, cz)
        mid2 = (cx + curve * 0.7, cy + self.beam_length * 0.7, cz)

        segs.moveTo(start)
        segs.drawTo(mid1)
        segs.drawTo(mid2)
        segs.drawTo(end)

        self.nav_np = self.nav_root.attachNewNode(segs.create())

    # =============================
    # 更新
    # =============================
    def update(self, task: Task):
        dt = globalClock.getDt()
        self.update_camera(dt)

        move = self.road_speed * dt

        for objs, limit in [(self.roads, -self.segment_length),
                             (self.lanes, -self.segment_length)]:
            for o in objs:
                o.setY(o.getY() - move)
                if o.getY() < limit:
                    o.setY(o.getY() + self.segment_length * 2)

        for pole in self.poles:
            pole.setY(pole.getY() - move)
            if pole.getY() < -10:
                pole.setY(pole.getY() + self.segment_length)

        for car in self.oncoming_cars:
            car.setY(car.getY() - move)
            if car.getY() < -20:
                car.setY(self.segment_length)

        x = self.car.getX()
        if self.move_left:
            x -= self.car_speed_x * dt
        if self.move_right:
            x += self.car_speed_x * dt

        dx = self.car.getX() - self.car1.getX()
        if self.auto_align:
            if abs(dx) > self.dead_zone:
                x -= dx * self.align_speed * dt
            else:
                x = self.car1.getX()
                self.auto_align = False

        self.car.setX(max(self.min_x, min(self.max_x, x)))

        if abs(dx) > self.dead_zone:
            if dx > 0:
                self.warning_text.setText("WARNING: Vehicle drifting to the Right")
                self.warning_text.setTextColor(0.8, 0.3, 0.3, 1)
            else:
                self.warning_text.setText("WARNING: Vehicle drifting to the Left")
                self.warning_text.setTextColor(0.3, 0.5, 0.9, 1)
            self.warning_np.show()
        else:
            self.warning_np.hide()

        self.update_nav_beam()
        return Task.cont


app = TeslaLikeDemo()
app.run()
